# Thinking1	 什么是近似最近邻查找，常用的方法有哪些?

答：ANN，就是离目标近似最近邻检索，牺牲精度的情况下提高搜索效率。
常用的方法有：MinHash和SimHash

# Thinking2  为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似    度

答：对于集合Ci与Cj，对应的行有三种可能：
                                    A类：两列的值都为1；
                                    B类：其中一列的值为0，另一列的值为1；
                                    C类：两列的值都为0.
C类行对于结果计算没有影响，可以删除，
P(h(Ci)=h(Cj))=P(删掉C类后，第一行为A类)=A类行的个数/所有行的个数=a/(a+b)
Jaccard(Ci,Cj) = (Ci,Cj)交集/(Ci,Cj)并集
所以，P(h(Ci)=h(Cj))= Jaccard(Ci,Cj)

# Thinking3	 SimHash在计算文档相似度的作用是怎样的？

Step1, 基于传统的IR方法，将文章转换为一组加权的特征值构成的向量
Step2，通过SimHash算法得到每篇文档的指纹（fingerprint）
Step3，计算不同文档之间的Haming距离




# Thinking4  为什么YouTube采用期望观看时间作为评估指标
答：从商业价值上考虑所以采用观看时间作为评估指标，因为观看时长可以插入广告，带来广告费用收入


#  Action1	
"使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt  针对某句话进行Query，查找Top-3相似的句子"


In [4]:
from datasketch import MinHash,MinHashLSHForest
import jieba.posseg as pseg
import re

In [11]:
# 读取文件
with open('./weibos.txt', 'r', encoding='UTF-8') as f:
    text = f.read()


In [21]:
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？]', text.replace(' \u200b\n', '').replace('\n', '').replace(' \u200b', ''))
print("sentences",sentences)

sentences ['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用', '国足输给叙利亚之后，里皮辞职', '谁将成为新主帅，成为广大球迷关注的焦点', '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里', '据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决', '据了解', '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战', '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利', '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话', '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商', '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……', '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足', '老头有点意思，凡是里皮干过的地方，他就想试试', '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛', '可以试试', '斯科拉里的水平，还不如里皮', '斯科拉里，看好的不是国足，而是年薪……非常应该辞职', '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有', '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了', '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光', '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队', '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去', '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛', '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州', '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事', '”中国足协：接受里

In [23]:
# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()
print("sentences",sentences)

sentences ['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用', '国足输给叙利亚之后，里皮辞职', '谁将成为新主帅，成为广大球迷关注的焦点', '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里', '据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决', '据了解', '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战', '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利', '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话', '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商', '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……', '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足', '老头有点意思，凡是里皮干过的地方，他就想试试', '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛', '可以试试', '斯科拉里的水平，还不如里皮', '斯科拉里，看好的不是国足，而是年薪……非常应该辞职', '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有', '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了', '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光', '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队', '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去', '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛', '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州', '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事', '”中国足协：接受里

In [39]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [24]:
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [35]:
# 设置停用词
stop = [line.strip() for line in open('stopwords.txt').readlines()]

In [36]:
print(stop)

['啊', '阿', '哎', '哎呀', '哎哟', '唉', '俺', '俺们', '按', '按照', '吧', '吧哒', '把', '罢了', '被', '本', '本着', '比', '比方', '比如', '鄙人', '彼', '彼此', '边', '别', '别的', '别说', '并', '并且', '不比', '不成', '不单', '不但', '不独', '不管', '不光', '不过', '不仅', '不拘', '不论', '不怕', '不然', '不如', '不特', '不惟', '不问', '不只', '朝', '朝着', '趁', '趁着', '乘', '冲', '除', '除此之外', '除非', '除了', '此', '此间', '此外', '从', '从而', '打', '待', '但', '但是', '当', '当着', '到', '得', '的', '的话', '等', '等等', '地', '第', '叮咚', '对', '对于', '多', '多少', '而', '而况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或者', '极了', '及', '及其', '及至', '即', '即便', '即或', '即令', '即若', '即使', '几', '几时', '己', '既', '既然', '既是', '继而', '加之', '假如', '假若', '假使', '鉴于', '将', '较', '较之', '叫', '接着', '结果', '紧接着', '进而', '尽', '尽管', '经', '经过', '就', '就是', '就是说', '据', 

In [40]:
# 分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmppvnczf5e' -> '/tmp/jieba.cache'
Loading model cost 1.324 seconds.
Prefix dict has been built successfully.


In [41]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)

In [42]:
# index所有key，以便可以进行检索
forest.index()


In [48]:
query = '换教练国足能进世界杯'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

0 0.2890625 斯科拉里愿意执教国足上一届如果是里皮从头芾尾是很大机会入世界杯一届没几个能用大力归化都没用
13 0.3203125 老头是世界杯冠军教头折中国没啥丢人毕竟里皮折嘛
31 0.15625 曾经以为世界名帅里皮能够带领国足走出迷茫国足蒸蒸日上我们国人不再迷茫吃下一颗定心丸
Top 3 邻居 [0, 13, 31]
